In [1]:
# Install required packages
!pip install -q langchain langchain_community pypdf nltk spacy transformers duckduckgo-search pandas matplotlib seaborn google-search-results python-docx pdfplumber
!python -m spacy download en_core_web_sm
!pip install nest_asyncio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('all')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downlo

True

In [6]:
import re
import os
import nltk
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import asyncio
import nest_asyncio
from google.colab import files
from datetime import datetime
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from collections import Counter, defaultdict
import pdfplumber
import docx
from duckduckgo_search import DDGS
from transformers import pipeline
import json

# Initialize NLP tools
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
spacy_nlp = spacy.load('en_core_web_sm')

# Enable nested asyncio for running async code in Colab
nest_asyncio.apply()

# Initialize sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

class RFPRequirementsExtractor:
    def __init__(self):
        # Define category keywords for requirement classification
        self.requirement_categories = {
            "Technical": [
                "technical", "technology", "software", "hardware", "system", "platform", "architecture",
                "database", "cloud", "infrastructure", "api", "integration", "interface", "security",
                "performance", "scalability", "availability", "reliability", "maintainability", "compatibility",
                "framework", "protocol", "encryption", "authentication", "authorization", "bandwidth",
                "storage", "server", "network", "device", "mobile", "responsive", "browser", "operating system",
                "backup", "recovery", "disaster recovery", "continuity", "monitoring", "log", "alert",
                "configuration", "deployment", "installation", "update", "upgrade", "patch", "version",
                "technology stack", "programming language", "code", "development", "testing", "quality assurance"
            ],
            "Legal": [
                "legal", "compliance", "regulation", "regulatory", "law", "legislation", "statute",
                "code", "act", "rule", "policy", "guideline", "standard", "certification", "accreditation",
                "iso", "soc", "hipaa", "gdpr", "ccpa", "ferpa", "pci", "dss", "sox", "copyright",
                "trademark", "patent", "intellectual property", "ip", "confidentiality", "privacy",
                "data protection", "liability", "indemnification", "indemnify", "warranty", "warranties",
                "remedies", "damages", "injunction", "arbitration", "mediation", "dispute", "jurisdiction",
                "governing law", "venue", "court", "litigation", "settlement", "termination", "contract",
                "agreement", "license", "permit", "consent", "authorization", "approval"
            ],
            "Financial": [
                "financial", "finance", "cost", "price", "pricing", "budget", "funding", "payment",
                "invoice", "billing", "fee", "expense", "expenditure", "disbursement", "reimbursement",
                "tax", "discount", "credit", "debit", "revenue", "profit", "margin", "roi", "return on investment",
                "tco", "total cost of ownership", "capital", "operational", "opex", "capex", "fixed cost",
                "variable cost", "direct cost", "indirect cost", "overhead", "depreciation", "amortization",
                "interest", "loan", "debt", "equity", "financing", "fund", "grant", "subsidy", "incentive",
                "penalty", "fine", "surcharge", "late fee", "early payment", "milestone payment", "installment",
                "retainer", "deposit", "performance bond", "letter of credit", "escrow", "insurance"
            ],
            "Timeline": [
                "timeline", "schedule", "date", "deadline", "due date", "delivery date", "milestone",
                "phase", "stage", "period", "duration", "interval", "frequency", "cycle", "iteration",
                "sprint", "release", "version", "calendar", "month", "week", "day", "hour", "minute",
                "time", "timing", "punctual", "delay", "expedite", "accelerate", "fast-track", "critical path",
                "dependency", "predecessor", "successor", "sequence", "concurrent", "parallel", "serial",
                "cascading", "overlap", "gap", "buffer", "float", "slack", "contingency", "extension",
                "postponement", "deferment", "advancement", "early", "late", "on-time", "timely"
            ],
            "Operational": [
                "operational", "operation", "process", "procedure", "workflow", "workstream", "business process",
                "service", "support", "maintenance", "administration", "management", "governance", "oversight",
                "supervision", "coordination", "collaboration", "communication", "reporting", "documentation",
                "manual", "guide", "handbook", "instruction", "direction", "specification", "requirement",
                "training", "education", "learning", "knowledge transfer", "onboarding", "implementation",
                "execution", "production", "delivery", "logistics", "supply chain", "inventory", "warehouse",
                "distribution", "transportation", "shipping", "receiving", "handling", "packaging", "storage"
            ],
            "Personnel": [
                "personnel", "staff", "employee", "worker", "contractor", "consultant", "vendor", "supplier",
                "partner", "associate", "resource", "team", "crew", "group", "department", "division",
                "organization", "company", "corporation", "firm", "agency", "institution", "establishment",
                "enterprise", "business", "entity", "role", "responsibility", "duty", "task", "function",
                "job", "position", "title", "rank", "grade", "level", "hierarchy", "structure", "skill",
                "capability", "competency", "qualification", "certification", "experience", "expertise",
                "knowledge", "ability", "proficiency", "mastery", "specialization", "specialty", "discipline"
            ],
            "Quality": [
                "quality", "standard", "specification", "requirement", "conformance", "compliance",
                "adherence", "audit", "review", "inspection", "testing", "verification", "validation",
                "acceptance", "rejection", "defect", "error", "bug", "issue", "problem", "failure",
                "malfunction", "nonconformity", "deviation", "variance", "tolerance", "threshold",
                "benchmark", "baseline", "metric", "measure", "indicator", "kpi", "key performance indicator",
                "sla", "service level agreement", "ola", "operational level agreement", "qos", "quality of service",
                "assurance", "control", "improvement", "enhancement", "optimization", "refinement"
            ],
            "Deliverables": [
                "deliverable", "output", "outcome", "result", "product", "service", "solution", "system",
                "application", "program", "software", "hardware", "device", "equipment", "tool", "utility",
                "module", "component", "element", "feature", "function", "capability", "report", "document",
                "documentation", "manual", "guide", "handbook", "specification", "design", "plan", "blueprint",
                "diagram", "model", "prototype", "proof of concept", "poc", "mvp", "minimum viable product",
                "alpha", "beta", "release", "version", "build", "package", "bundle", "kit", "set", "collection"
            ],
            "Environmental": [
                "environmental", "environment", "sustainability", "sustainable", "green", "eco-friendly",
                "ecological", "eco", "carbon", "emission", "pollution", "waste", "recycling", "reuse",
                "reduce", "conservation", "preservation", "protection", "restoration", "remediation",
                "mitigation", "adaptation", "resilience", "climate", "weather", "temperature", "humidity",
                "pressure", "air", "water", "soil", "earth", "land", "terrain", "geography", "geology",
                "biodiversity", "ecosystem", "habitat", "flora", "fauna", "wildlife", "natural", "nature",
                "resource", "energy", "power", "electricity", "fuel", "gas", "oil", "coal", "renewable"
            ]
        }

        # Define priority markers with their weights
        self.priority_markers = {
            "must": 10, "shall": 10, "required": 10, "mandatory": 10, "essential": 10, "critical": 10,
            "necessary": 8, "important": 8, "significant": 8, "should": 6, "recommended": 6,
            "desired": 4, "preferred": 4, "may": 2, "optional": 2, "nice to have": 2
        }

        # Define common requirement section headers - without inline flags
        self.requirement_section_patterns = [
            r"requirements?",
            r"technical\s+requirements?",
            r"functional\s+requirements?",
            r"non-?functional\s+requirements?",
            r"statement\s+of\s+work",
            r"scope\s+of\s+work",
            r"specifications?",
            r"deliverables?",
            r"compliance(\s+requirements?)?",
            r"minimum\s+qualifications?",
            r"minimum\s+requirements?",
            r"legal\s+requirements?",
            r"technical\s+specifications?",
            r"service\s+level\s+agreements?",
            r"acceptance\s+criteria",
            r"performance\s+standards?",
            r"security\s+requirements?",
            r"data\s+protection\s+requirements?"
        ]

    async def upload_rfp_document(self):
        """Upload an RFP document in PDF or DOCX format."""
        print("Please upload the RFP document (PDF or DOCX format)...")
        uploaded = files.upload()

        if not uploaded:
            print("No file was uploaded.")
            return None

        # Get the filename of the uploaded file
        filename = list(uploaded.keys())[0]
        return filename

    def extract_text_from_document(self, filename):
        """Extract text from PDF or DOCX file."""
        if filename.lower().endswith('.pdf'):
            return self.extract_text_from_pdf(filename)
        elif filename.lower().endswith('.docx'):
            return self.extract_text_from_docx(filename)
        else:
            print(f"Unsupported file format: {filename}")
            return ""

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from PDF using pdfplumber."""
        try:
            text = ""
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text += page.extract_text() or ""
                    text += "\n\n"
            return text
        except Exception as e:
            print(f"Error extracting text from PDF: {str(e)}")
            return ""

    def extract_text_from_docx(self, docx_path):
        """Extract text from DOCX file."""
        try:
            doc = docx.Document(docx_path)
            text = ""
            for para in doc.paragraphs:
                text += para.text + "\n"
            return text
        except Exception as e:
            print(f"Error extracting text from DOCX: {str(e)}")
            return ""

    def extract_requirements(self, text):
        """Extract requirements from the text by identifying sentences that contain requirement patterns."""
        # Split text into sentences
        sentences = sent_tokenize(text)

        # Initialize requirements list
        requirements = []

        # Keep track of the current section
        current_section = "General"

        # Process each sentence
        for i, sentence in enumerate(sentences):
            # Check if this sentence starts a new section
            for pattern in self.requirement_section_patterns:
                if re.search(f"^{pattern}", sentence.strip(), re.IGNORECASE):
                    current_section = sentence.strip()
                    break

            # Look for sentences that might be requirements
            if (
                any(re.search(r'\b' + re.escape(marker) + r'\b', sentence.lower()) for marker in self.priority_markers.keys()) or
                re.search(r'\b(provide|deliver|support|ensure|maintain|include|implement|develop|create|establish)\b', sentence.lower()) or
                re.search(r'\b(requirement|specification|mandatory|compliance|conform|adhere)\b', sentence.lower()) or
                re.search(r'[0-9]+\.[0-9]+\s', sentence) or  # Numbered requirements like "3.2 The system shall..."
                re.search(r'•|\*|-', sentence)  # Bullet points often indicate requirements
            ):
                # Process the requirement
                requirement = self.process_requirement(sentence, current_section)
                if requirement:
                    requirements.append(requirement)

        return requirements

    def process_requirement(self, sentence, section):
        """Process a potential requirement sentence to extract relevant information."""
        # Skip very short sentences
        if len(sentence.split()) < 3:
            return None

        # Clean up the sentence
        sentence = sentence.strip()

        # Determine priority based on terms used
        priority_score = 0
        priority_terms = []

        for marker, weight in self.priority_markers.items():
            if re.search(r'\b' + re.escape(marker) + r'\b', sentence.lower()):
                priority_score += weight
                priority_terms.append(marker)

        # If no priority markers found, use a default
        if priority_score == 0:
            priority_score = 5  # Default mid-level priority

        # Normalize priority score (0-10 scale)
        normalized_priority = min(10, priority_score)

        # Determine category
        categories = []
        for category, keywords in self.requirement_categories.items():
            if any(re.search(r'\b' + re.escape(keyword) + r'\b', sentence.lower()) for keyword in keywords):
                categories.append(category)

        # If no category was identified, use the section or "General"
        if not categories:
            if any(re.search(pattern, section, re.IGNORECASE) for pattern in self.requirement_section_patterns):
                # Extract category from section name
                section_lower = section.lower()
                if "technical" in section_lower:
                    categories.append("Technical")
                elif "legal" in section_lower or "compliance" in section_lower:
                    categories.append("Legal")
                elif "financial" in section_lower or "cost" in section_lower or "price" in section_lower:
                    categories.append("Financial")
                elif "timeline" in section_lower or "schedule" in section_lower:
                    categories.append("Timeline")
                elif "operational" in section_lower or "process" in section_lower:
                    categories.append("Operational")
                elif "personnel" in section_lower or "staff" in section_lower:
                    categories.append("Personnel")
                elif "quality" in section_lower:
                    categories.append("Quality")
                elif "deliverable" in section_lower:
                    categories.append("Deliverables")
                elif "environment" in section_lower:
                    categories.append("Environmental")
                else:
                    categories.append("General")
            else:
                categories.append("General")

        # Use spaCy to identify entities
        doc = spacy_nlp(sentence)
        entities = [ent.text for ent in doc.ents]

        # Get sentiment
        try:
            sentiment = sentiment_analyzer(sentence[:512])[0]  # Limit length to avoid model errors
        except:
            sentiment = {"label": "NEUTRAL", "score": 0.5}

        return {
            "text": sentence,
            "section": section,
            "categories": categories,
            "priority": normalized_priority,
            "priority_terms": priority_terms,
            "entities": entities,
            "sentiment": sentiment
        }

    async def search_web_for_context(self, requirement, max_results=2):
        """Search the web for context about a specific requirement."""
        # Skip web search for requirements with low priority to prevent rate limiting
        if requirement["priority"] < 6:
            return "No additional context (low priority requirement)."

        try:
            # Create search query based on requirement
            categories = ", ".join(requirement["categories"])
            search_query = f"{categories} requirement: {requirement['text']}"

            # Limit query length
            search_query = search_query[:100]

            with DDGS() as ddgs:
                results = list(ddgs.text(search_query, max_results=max_results))

            if not results:
                return "No search results found."

            # Format results
            formatted_results = []
            for result in results:
                title = result.get('title', 'No Title')
                snippet = result.get('body', 'No content available')
                formatted_results.append(f"{title}: {snippet}")

            return "\n\n".join(formatted_results)
        except Exception as e:
            print(f"Search error: {str(e)}")
            return "Web search unavailable."

    async def enrich_requirements(self, requirements):
        """Enrich requirements with web context."""
        enriched_requirements = []

        print("Enriching requirements with web context...")

        # Process only high priority requirements to avoid rate limiting
        high_priority_reqs = [r for r in requirements if r["priority"] >= 8]
        medium_priority_reqs = [r for r in requirements if 5 <= r["priority"] < 8]
        low_priority_reqs = [r for r in requirements if r["priority"] < 5]

        # Process a sample of each priority level to avoid overwhelming the API
        sample_high = high_priority_reqs[:min(10, len(high_priority_reqs))]
        sample_medium = medium_priority_reqs[:min(5, len(medium_priority_reqs))]
        sample_low = low_priority_reqs[:min(2, len(low_priority_reqs))]

        # Combine the samples
        samples = sample_high + sample_medium + sample_low

        for i, req in enumerate(requirements):
            # Only search web for requirements in our sample
            if req in samples:
                print(f"Searching for context on requirement {i+1}/{len(requirements)}...")
                web_context = await self.search_web_for_context(req)
                req["web_context"] = web_context
            else:
                req["web_context"] = "Web search skipped for efficiency."

            enriched_requirements.append(req)

        return enriched_requirements

    def prioritize_requirements(self, requirements):
        """Sort requirements by priority and category."""
        # Create a mapping of categories to importance levels
        category_importance = {
            "Legal": 10,      # Highest importance - legal requirements are usually non-negotiable
            "Technical": 9,
            "Security": 8,
            "Financial": 7,
            "Quality": 6,
            "Timeline": 5,
            "Deliverables": 4,
            "Personnel": 3,
            "Operational": 2,
            "Environmental": 1,
            "General": 0      # Lowest importance
        }

        # Sort requirements by priority and then by category importance
        def requirement_sort_key(req):
            # Get the highest importance category for this requirement
            max_category_importance = max(
                [category_importance.get(cat, 0) for cat in req["categories"]],
                default=0
            )

            # Return a tuple of (priority, category_importance) for sorting
            return (-req["priority"], -max_category_importance)

        return sorted(requirements, key=requirement_sort_key)

    def analyze_requirements(self, requirements):
        """Analyze requirements to generate insights."""
        # Initialize analysis result
        analysis = {
            "total_requirements": len(requirements),
            "priority_distribution": {
                "high": len([r for r in requirements if r["priority"] >= 8]),
                "medium": len([r for r in requirements if 5 <= r["priority"] < 8]),
                "low": len([r for r in requirements if r["priority"] < 5])
            },
            "category_distribution": defaultdict(int),
            "key_entities": Counter(),
            "section_distribution": defaultdict(int),
            "potential_conflicts": [],
            "potential_dependencies": [],
            "sentiment_analysis": {
                "positive": 0,
                "negative": 0,
                "neutral": 0
            }
        }

        # Process requirements for analysis
        for req in requirements:
            # Count categories
            for category in req["categories"]:
                analysis["category_distribution"][category] += 1

            # Count entities
            for entity in req["entities"]:
                analysis["key_entities"][entity] += 1

            # Count sections
            analysis["section_distribution"][req["section"]] += 1

            # Sentiment analysis
            sentiment = req.get("sentiment", {}).get("label", "NEUTRAL")
            if sentiment == "POSITIVE":
                analysis["sentiment_analysis"]["positive"] += 1
            elif sentiment == "NEGATIVE":
                analysis["sentiment_analysis"]["negative"] += 1
            else:
                analysis["sentiment_analysis"]["neutral"] += 1

        # Identify potential conflicts (requirements with similar entities but different categories)
        entity_to_reqs = defaultdict(list)
        for i, req in enumerate(requirements):
            for entity in req["entities"]:
                if len(entity) > 3:  # Ignore very short entities
                    entity_to_reqs[entity.lower()].append(i)

        for entity, req_indices in entity_to_reqs.items():
            if len(req_indices) > 1:
                req_categories = set()
                for idx in req_indices:
                    for cat in requirements[idx]["categories"]:
                        req_categories.add(cat)

                if len(req_categories) > 1:
                    analysis["potential_conflicts"].append({
                        "entity": entity,
                        "requirement_indices": req_indices,
                        "categories": list(req_categories)
                    })

        # Identify potential dependencies (requirements in one category that mention entities in another)
        for i, req in enumerate(requirements):
            for entity in req["entities"]:
                if len(entity) > 3:  # Ignore very short entities
                    for j, other_req in enumerate(requirements):
                        if i != j and entity.lower() in other_req["text"].lower():
                            # Check if categories are different
                            req_cats = set(req["categories"])
                            other_cats = set(other_req["categories"])
                            if req_cats != other_cats:
                                analysis["potential_dependencies"].append({
                                    "entity": entity,
                                    "source_index": i,
                                    "dependent_index": j,
                                    "source_categories": list(req_cats),
                                    "dependent_categories": list(other_cats)
                                })

        # Limit the number of conflicts and dependencies to report
        analysis["potential_conflicts"] = analysis["potential_conflicts"][:10]
        analysis["potential_dependencies"] = analysis["potential_dependencies"][:10]

        # Convert counters to dictionaries for JSON serialization
        analysis["category_distribution"] = dict(analysis["category_distribution"])
        analysis["section_distribution"] = dict(analysis["section_distribution"])
        analysis["key_entities"] = dict(analysis["key_entities"].most_common(20))

        return analysis

    def format_requirements_as_dataframe(self, requirements):
        """Format requirements as a pandas DataFrame for easier viewing and manipulation."""
        df_data = []
        for req in requirements:
            # Create a row for each requirement
            row = {
                "Requirement": req["text"],
                "Priority": req["priority"],
                "Categories": ", ".join(req["categories"]),
                "Section": req["section"],
                "Priority Terms": ", ".join(req["priority_terms"]) if req["priority_terms"] else "None",
                "Entities": ", ".join(req["entities"]) if req["entities"] else "None",
                "Sentiment": req.get("sentiment", {}).get("label", "NEUTRAL")
            }
            df_data.append(row)

        return pd.DataFrame(df_data)

    def visualize_requirements(self, requirements, analysis):
        """Generate visualizations for requirements analysis."""
        # Create figure with subplots
        fig, axs = plt.subplots(2, 2, figsize=(18, 14))

        # 1. Priority distribution pie chart
        priority_labels = ['High (8-10)', 'Medium (5-7)', 'Low (1-4)']
        priority_sizes = [
            analysis["priority_distribution"]["high"],
            analysis["priority_distribution"]["medium"],
            analysis["priority_distribution"]["low"]
        ]
        priority_colors = ['#ff6666', '#ffcc99', '#99cc99']

        axs[0, 0].pie(priority_sizes, labels=priority_labels, colors=priority_colors, autopct='%1.1f%%', startangle=90)
        axs[0, 0].set_title('Requirements by Priority', fontsize=14)

        # 2. Category distribution bar chart
        categories = list(analysis["category_distribution"].keys())
        category_counts = list(analysis["category_distribution"].values())

        # Sort by count
        sorted_indices = np.argsort(category_counts)
        sorted_categories = [categories[i] for i in sorted_indices]
        sorted_counts = [category_counts[i] for i in sorted_indices]

        # Plot horizontal bar chart
        axs[0, 1].barh(sorted_categories, sorted_counts, color='#6699cc')
        axs[0, 1].set_title('Requirements by Category', fontsize=14)
        axs[0, 1].set_xlabel('Number of Requirements')

        # 3. Top entities bar chart
        if analysis["key_entities"]:
            top_entities = list(analysis["key_entities"].keys())[:10]
            entity_counts = [analysis["key_entities"][e] for e in top_entities]

            axs[1, 0].bar(top_entities, entity_counts, color='#cc99cc')
            axs[1, 0].set_title('Top Entities Mentioned', fontsize=14)
            axs[1, 0].set_ylabel('Frequency')
            plt.setp(axs[1, 0].get_xticklabels(), rotation=45, ha='right')
        else:
            axs[1, 0].text(0.5, 0.5, 'No entities found', ha='center', va='center', fontsize=12)
            axs[1, 0].set_title('Top Entities Mentioned', fontsize=14)

        # 4. Sentiment analysis
        sentiment_labels = ['Positive', 'Neutral', 'Negative']
        sentiment_sizes = [
            analysis["sentiment_analysis"]["positive"],
            analysis["sentiment_analysis"]["neutral"],
            analysis["sentiment_analysis"]["negative"]
        ]
        sentiment_colors = ['#99cc99', '#cccccc', '#ff6666']

        axs[1, 1].pie(sentiment_sizes, labels=sentiment_labels, colors=sentiment_colors, autopct='%1.1f%%', startangle=90)
        axs[1, 1].set_title('Sentiment of Requirements', fontsize=14)

        plt.tight_layout()
        plt.savefig('requirements_analysis.png')
        plt.close()

        # Create a heatmap of priority by category
        priority_by_category = defaultdict(list)

        for req in requirements:
            for category in req["categories"]:
                priority_by_category[category].append(req["priority"])

        # Calculate average priority for each category
        category_avg_priority = {}
        for category, priorities in priority_by_category.items():
            if priorities:
                category_avg_priority[category] = sum(priorities) / len(priorities)

        if category_avg_priority:
            # Create a new DataFrame for the heatmap
            categories = list(category_avg_priority.keys())
            avg_priorities = [category_avg_priority[cat] for cat in categories]

            # Sort by average priority
            sorted_indices = np.argsort(avg_priorities)[::-1]  # Descending
            sorted_categories = [categories[i] for i in sorted_indices]
            sorted_priorities = [avg_priorities[i] for i in sorted_indices]

            heatmap_data = pd.DataFrame({
                'Category': sorted_categories,
                'Average Priority': sorted_priorities
            })

            plt.figure(figsize=(12, 8))
            heatmap = sns.heatmap(
                heatmap_data.set_index('Category')['Average Priority'].to_frame().T,
                annot=True,
                cmap="YlOrRd",
                fmt=".2f",
                linewidths=.5
            )
            plt.title('Average Priority by Category', fontsize=16)
            plt.savefig('priority_heatmap.png')
            plt.close()

        return 'requirements_analysis.png', 'priority_heatmap.png'

    def generate_html_report(self, requirements, analysis, visualizations):
        """Generate an HTML report for the requirements analysis."""
        # Prepare the HTML content
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>RFP Requirements Analysis Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; max-width: 1200px; margin: 0 auto; padding: 20px; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                table {{ border-collapse: collapse; width: 100%; margin-bottom: 20px; }}
                th, td {{ text-align: left; padding: 12px; }}
                th {{ background-color: #2c3e50; color: white; }}
                tr:nth-child(even) {{ background-color: #f2f2f2; }}
                tr:hover {{ background-color: #ddd; }}
                .high-priority {{ background-color: #ffcccc; }}
                .medium-priority {{ background-color: #ffffcc; }}
                .low-priority {{ background-color: #ccffcc; }}
                .requirement-section {{ background-color: #eef2f7; padding: 15px; margin: 15px 0; border-left: 5px solid #2c3e50; }}
                .insights {{ background-color: #eef7f2; padding: 15px; margin: 15px 0; border-left: 5px solid #27ae60; }}
                .visualizations img {{ max-width: 100%; height: auto; margin: 10px 0; }}
                .header {{ display: flex; justify-content: space-between; align-items: center; }}
                .header-info {{ text-align: right; }}
                .toc {{ background-color: #f8f9fa; padding: 20px; margin-bottom: 20px; }}
                .toc ol {{ padding-left: 20px; }}
                .toc a {{ text-decoration: none; color: #3498db; }}
                .footer {{ text-align: center; margin-top: 30px; padding-top: 20px; border-top: 1px solid #eee; color: #7f8c8d; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>RFP Requirements Analysis Report</h1>
                <div class="header-info">
                    <p>Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>
                </div>
            </div>

            <div class="toc">
                <h2>Table of Contents</h2>
                <ol>
                    <li><a href="#executive-summary">Executive Summary</a></li>
                    <li><a href="#visualizations">Visualizations</a></li>
                    <li><a href="#high-priority">High Priority Requirements</a></li>
                    <li><a href="#medium-priority">Medium Priority Requirements</a></li>
                    <li><a href="#low-priority">Low Priority Requirements</a></li>
                    <li><a href="#category-analysis">Requirements by Category</a></li>
                    <li><a href="#potential-issues">Potential Issues & Dependencies</a></li>
                </ol>
            </div>

            <section id="executive-summary" class="requirement-section">
                <h2>Executive Summary</h2>
                <p>This analysis identified <strong>{analysis["total_requirements"]}</strong> distinct requirements in the RFP document.</p>
                <ul>
                    <li><strong>High Priority Requirements:</strong> {analysis["priority_distribution"]["high"]} ({analysis["priority_distribution"]["high"]/analysis["total_requirements"]*100:.1f}%)</li>
                    <li><strong>Medium Priority Requirements:</strong> {analysis["priority_distribution"]["medium"]} ({analysis["priority_distribution"]["medium"]/analysis["total_requirements"]*100:.1f}%)</li>
                    <li><strong>Low Priority Requirements:</strong> {analysis["priority_distribution"]["low"]} ({analysis["priority_distribution"]["low"]/analysis["total_requirements"]*100:.1f}%)</li>
                </ul>
                <p><strong>Primary Requirement Categories:</strong></p>
                <ul>
        """

        # Add top categories to the summary
        sorted_categories = sorted(analysis["category_distribution"].items(), key=lambda x: x[1], reverse=True)
        for category, count in sorted_categories[:5]:
            percentage = count / analysis["total_requirements"] * 100
            html_content += f"<li>{category}: {count} ({percentage:.1f}%)</li>"

        html_content += """
                </ul>
            </section>

            <section id="visualizations" class="visualizations">
                <h2>Visualizations</h2>
        """

        # Add visualizations
        for viz_path in visualizations:
            html_content += f'<img src="{viz_path}" alt="Requirements Analysis Visualization">'

        html_content += """
            </section>

            <section id="high-priority" class="requirement-section">
                <h2>High Priority Requirements</h2>
                <p>The following requirements are considered high priority (8-10) and should be addressed first:</p>
                <table>
                    <tr>
                        <th>Requirement</th>
                        <th>Priority</th>
                        <th>Categories</th>
                        <th>Section</th>
                    </tr>
        """

        # Add high priority requirements
        high_priority_reqs = [r for r in requirements if r["priority"] >= 8]
        for req in high_priority_reqs:
            html_content += f"""
                <tr class="high-priority">
                    <td>{req["text"]}</td>
                    <td>{req["priority"]}</td>
                    <td>{', '.join(req["categories"])}</td>
                    <td>{req["section"]}</td>
                </tr>
            """

        if not high_priority_reqs:
            html_content += """
                <tr>
                    <td colspan="4">No high priority requirements identified.</td>
                </tr>
            """

        html_content += """
                </table>
            </section>

            <section id="medium-priority" class="requirement-section">
                <h2>Medium Priority Requirements</h2>
                <p>The following requirements are considered medium priority (5-7):</p>
                <table>
                    <tr>
                        <th>Requirement</th>
                        <th>Priority</th>
                        <th>Categories</th>
                        <th>Section</th>
                    </tr>
        """

        # Add medium priority requirements
        medium_priority_reqs = [r for r in requirements if 5 <= r["priority"] < 8]
        for req in medium_priority_reqs:
            html_content += f"""
                <tr class="medium-priority">
                    <td>{req["text"]}</td>
                    <td>{req["priority"]}</td>
                    <td>{', '.join(req["categories"])}</td>
                    <td>{req["section"]}</td>
                </tr>
            """

        if not medium_priority_reqs:
            html_content += """
                <tr>
                    <td colspan="4">No medium priority requirements identified.</td>
                </tr>
            """

        html_content += """
                </table>
            </section>

            <section id="low-priority" class="requirement-section">
                <h2>Low Priority Requirements</h2>
                <p>The following requirements are considered lower priority (1-4):</p>
                <table>
                    <tr>
                        <th>Requirement</th>
                        <th>Priority</th>
                        <th>Categories</th>
                        <th>Section</th>
                    </tr>
        """

        # Add low priority requirements
        low_priority_reqs = [r for r in requirements if r["priority"] < 5]
        for req in low_priority_reqs:
            html_content += f"""
                <tr class="low-priority">
                    <td>{req["text"]}</td>
                    <td>{req["priority"]}</td>
                    <td>{', '.join(req["categories"])}</td>
                    <td>{req["section"]}</td>
                </tr>
            """

        if not low_priority_reqs:
            html_content += """
                <tr>
                    <td colspan="4">No low priority requirements identified.</td>
                </tr>
            """

        html_content += """
                </table>
            </section>

            <section id="category-analysis" class="requirement-section">
                <h2>Requirements by Category</h2>
        """

        # Add requirements by category
        for category, count in sorted_categories:
            if count > 0:
                html_content += f"""
                    <h3>{category} Requirements ({count})</h3>
                    <table>
                        <tr>
                            <th>Requirement</th>
                            <th>Priority</th>
                            <th>Section</th>
                        </tr>
                """

                category_reqs = [r for r in requirements if category in r["categories"]]
                for req in category_reqs:
                    priority_class = "high-priority" if req["priority"] >= 8 else ("medium-priority" if req["priority"] >= 5 else "low-priority")
                    html_content += f"""
                        <tr class="{priority_class}">
                            <td>{req["text"]}</td>
                            <td>{req["priority"]}</td>
                            <td>{req["section"]}</td>
                        </tr>
                    """

                html_content += """
                    </table>
                """

        html_content += """
            </section>

            <section id="potential-issues" class="insights">
                <h2>Potential Issues & Dependencies</h2>
        """

        # Add potential conflicts
        if analysis["potential_conflicts"]:
            html_content += """
                <h3>Potential Conflicts</h3>
                <p>The following requirements may have conflicting goals or interpretations:</p>
                <ul>
            """

            for conflict in analysis["potential_conflicts"]:
                html_content += f"""
                    <li>
                        <strong>Entity: {conflict["entity"]}</strong><br>
                        Appears in {len(conflict["requirement_indices"])} requirements across categories: {', '.join(conflict["categories"])}<br>
                        <ul>
                """

                for idx in conflict["requirement_indices"]:
                    req = requirements[idx]
                    html_content += f"""
                        <li>{req["text"]} (Priority: {req["priority"]})</li>
                    """

                html_content += """
                        </ul>
                    </li>
                """

            html_content += """
                </ul>
            """
        else:
            html_content += """
                <h3>Potential Conflicts</h3>
                <p>No significant conflicts identified.</p>
            """

        # Add potential dependencies
        if analysis["potential_dependencies"]:
            html_content += """
                <h3>Potential Dependencies</h3>
                <p>The following requirements may have dependencies on each other:</p>
                <ul>
            """

            for dependency in analysis["potential_dependencies"]:
                source_req = requirements[dependency["source_index"]]
                dependent_req = requirements[dependency["dependent_index"]]

                html_content += f"""
                    <li>
                        <strong>Entity: {dependency["entity"]}</strong><br>
                        <strong>Source:</strong> {source_req["text"]} (Priority: {source_req["priority"]})<br>
                        <strong>Dependent:</strong> {dependent_req["text"]} (Priority: {dependent_req["priority"]})
                    </li>
                """

            html_content += """
                </ul>
            """
        else:
            html_content += """
                <h3>Potential Dependencies</h3>
                <p>No significant dependencies identified.</p>
            """

        # Add web search insights for top requirements
        html_content += """
            <h3>Additional Context from Web Search</h3>
            <p>For key requirements, additional context was gathered from web searches:</p>
        """

        web_context_reqs = [r for r in requirements if r.get("web_context") and r.get("web_context") != "Web search skipped for efficiency." and r.get("web_context") != "No additional context (low priority requirement)."]

        if web_context_reqs:
            html_content += "<ul>"
            for req in web_context_reqs[:5]:  # Show top 5 for brevity
                html_content += f"""
                    <li>
                        <strong>Requirement:</strong> {req["text"]}<br>
                        <strong>Web Context:</strong> {req["web_context"][:500]}...
                    </li>
                """
            html_content += "</ul>"
        else:
            html_content += "<p>No significant web context retrieved for high-priority requirements.</p>"

        html_content += """
            </section>

            <div class="footer">
                <p>Generated by RFP Requirements Extractor and Analyzer</p>
                <p>This report is for informational purposes only and should be reviewed by subject matter experts.</p>
            </div>
        </body>
        </html>
        """

        # Save the HTML report
        with open("rfp_requirements_analysis.html", "w", encoding="utf-8") as f:
            f.write(html_content)

        return "rfp_requirements_analysis.html"

    async def run_analysis(self):
        """Run the complete analysis pipeline."""
        print("=== RFP Requirements Extractor and Analyzer ===")

        # Step 1: Upload document
        filename = await self.upload_rfp_document()
        if not filename:
            return

        # Step 2: Extract text
        print(f"Extracting text from {filename}...")
        text = self.extract_text_from_document(filename)

        if not text:
            print("Failed to extract text from the document.")
            return

        print(f"Extracted {len(text)} characters of text.")

        # Step 3: Extract requirements
        print("Extracting requirements...")
        requirements = self.extract_requirements(text)

        if not requirements:
            print("No requirements identified in the document.")
            return

        print(f"Extracted {len(requirements)} potential requirements.")

        # Step 4: Enrich requirements with web context
        requirements = await self.enrich_requirements(requirements)

        # Step 5: Prioritize requirements
        print("Prioritizing requirements...")
        prioritized_requirements = self.prioritize_requirements(requirements)

        # Step 6: Analyze requirements
        print("Analyzing requirements...")
        analysis = self.analyze_requirements(prioritized_requirements)

        # Step 7: Generate visualizations
        print("Generating visualizations...")
        visualizations = self.visualize_requirements(prioritized_requirements, analysis)

        # Step 8: Generate HTML report
        print("Generating HTML report...")
        report_path = self.generate_html_report(prioritized_requirements, analysis, visualizations)

        # Step 9: Display a summary and export options
        print("\n=== RFP Requirements Analysis Complete ===")
        print(f"Total requirements identified: {len(requirements)}")
        print(f"High priority requirements: {analysis['priority_distribution']['high']}")
        print(f"Medium priority requirements: {analysis['priority_distribution']['medium']}")
        print(f"Low priority requirements: {analysis['priority_distribution']['low']}")

        # Create a pandas DataFrame for display in Colab
        df = self.format_requirements_as_dataframe(prioritized_requirements)

        # Display the DataFrame
        display(df.head(10))

        print(f"\nFull report saved to: {report_path}")
        print("Downloading report...")

        # Make files available for download
        files.download(report_path)
        for viz_path in visualizations:
            files.download(viz_path)

        # Export to Excel if requested
        df.to_excel("rfp_requirements.xlsx", index=False)
        files.download("rfp_requirements.xlsx")

        # In the run_analysis method, add this before the final return statement
        json_output_path = self.export_requirements_to_json(prioritized_requirements, analysis, visualizations)
        print(f"\nJSON file saved to: {json_output_path}")

        return prioritized_requirements, analysis, report_path

# Initialize and run the extractor
async def main():
    extractor = RFPRequirementsExtractor()
    await extractor.run_analysis()

# Run the main function
asyncio.run(main())

Device set to use cpu


=== RFP Requirements Extractor and Analyzer ===
Please upload the RFP document (PDF or DOCX format)...


Saving ELIGIBLE RFP - 1.pdf to ELIGIBLE RFP - 1 (1).pdf
Extracting text from ELIGIBLE RFP - 1 (1).pdf...


Extracted 55788 characters of text.
Extracting requirements...
Extracted 209 potential requirements.
Enriching requirements with web context...
Searching for context on requirement 1/209...
Searching for context on requirement 2/209...
Searching for context on requirement 3/209...
Searching for context on requirement 4/209...
Searching for context on requirement 5/209...
Searching for context on requirement 6/209...
Searching for context on requirement 9/209...
Searching for context on requirement 16/209...
Searching for context on requirement 18/209...
Searching for context on requirement 20/209...
Searching for context on requirement 21/209...
Searching for context on requirement 22/209...
Searching for context on requirement 24/209...
Search error: https://html.duckduckgo.com/html 202 Ratelimit
Searching for context on requirement 25/209...
Search error: https://lite.duckduckgo.com/lite/ 202 Ratelimit
Searching for context on requirement 27/209...
Search error: https://html.duckduck

Requirement  Priority  \
0  Anticipated Award Date Contingent upon Committ...        10   
1  Contractor shall comply with all State, Federa...        10   
2  To the extent permissible by the law, Contract...        10   
3  d. Invoices must have the correct contract rat...        10   
4  If a Contractor’s Hourly Pay Rate to its worke...        10   
5  In the event that\nMHMR needs those temporary ...        10   
6  MHMR shall have the right to reject proposals ...        10   
7  Any confidential or proprietary information sh...        10   
8  RFP 25-008 Temporary Staffing Services Page 16...        10   
9  as\nrequired by authorities having jurisdictio...        10   

                   Categories                              Section  \
0  Legal, Timeline, Personnel                              General   
1            Legal, Personnel  Specifications and Requirements\n1.   
2            Legal, Personnel  Specifications and Requirements\n1.   
3  Legal, Timeline, Personnel  Specifications and Requirements\n1.   
4            Legal, Personnel  Specifications and Requirements\n1.   
5            Legal, Personnel  Specifications and Requirements\n1.   
6            Legal, Personnel  Specifications and Requirements\n1.   
7            Legal, Personnel  Specifications and Requirements\n1.   
8            Legal, Personnel  Specifications and Requirements\n1.   
9                       Legal  Specifications and Requirements\n1.   

  Priority Terms                                           Entities Sentiment  
0           must  Anticipated Award Date Contingent, Committee a...  NEGATIVE  
1          shall  State, Local Laws, Americans, Title VII of the...  POSITIVE  
2          shall                                               MHMR  NEGATIVE  
3           must                             d. Invoices, each week  NEGATIVE  
4       required      MHMR, MHMR, the Contractor’s Hourly Bill Rate  NEGATIVE  
5  required, may                                   MHMR, MHMR, MHMR  NEGATIVE  
6          shall                                               MHMR  NEGATIVE  
7          shall            MHMR, the Texas Public\nInformation Act  NEGATIVE  
8           must                                         25, 16, 33  NEGATIVE  
9       required                                               None  POSITIVE

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

AttributeError: 'RFPRequirementsExtractor' object has no attribute 'export_requirements_to_json'

In [9]:
def export_requirements_to_json(self, requirements, analysis, visualizations, output_filename="requirements_analysis.json"):
    """Export the requirements analysis to a JSON file."""

    # Get current date and time in ISO format
    current_time = datetime.now().isoformat()

    # Structure the JSON data
    output_data = {
        "metadata": {
            "generated_date": current_time,
            "source_document": getattr(self, 'source_document', os.path.basename(getattr(self, 'filename', "rfp_document.pdf"))),
            "total_requirements": len(requirements)
        },
        "requirements": [],
        "analysis": {
            "priority_distribution": analysis["priority_distribution"],
            "category_distribution": analysis["category_distribution"],
            "key_entities": analysis["key_entities"],
            "potential_conflicts": analysis["potential_conflicts"],
            "potential_dependencies": analysis["potential_dependencies"]
        },
        "visualizations": {
            "requirements_analysis_path": visualizations[0],
            "priority_heatmap_path": visualizations[1]
        }
    }

    # Add each requirement to the output data
    for i, req in enumerate(requirements):
        req_data = {
            "id": i + 1,
            "text": req["text"],
            "priority": req["priority"],
            "priority_terms": req["priority_terms"],
            "categories": req["categories"],
            "section": req["section"],
            "entities": req["entities"],
            "sentiment": req.get("sentiment", {"label": "NEUTRAL", "score": 0.5}),
            "web_context": req.get("web_context", "")
        }
        output_data["requirements"].append(req_data)

    # Write the JSON file
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=2)

    print(f"Requirements analysis exported to {output_filename}")

    # Make the file available for download in Colab
    files.download(output_filename)

    return output_filename